In [ ]:
import numpy as np

from Common_Functions.CommonFunctions import *
from Patches_Functions.ExtractPatches import extract_patches
from sklearn.neighbors import NearestNeighbors
# from color_transfer import color_transfer
# from irls import irls
# from segmentation.FaceDetectionSegmentation import segement
import cv2
from skimage.transform import pyramid_gaussian
import skimage.filters as f
from scipy.ndimage import gaussian_filter
import imageio.v3 as iio

In [ ]:
def irls(X,z,r,Iirls,patch_size,subsampling_gap):
    Xp=extract_patches(X,patch_size,subsampling_gap)
    num_of_patches=(np.shape(Xp)[0])*(np.shape(Xp)[1])
    w=(np.ones((num_of_patches,1)))
    # z should be initialized with patch matching
    print(num_of_patches)
    for i in range(0, Iirls):
        for x in range(0, np.shape(Xp)[0]):
            for y in range(0,np.shape(Xp)[1]):
                e=Xp[x,y,0,:,:,:]-z[x*np.shape(Xp)[1]+y]
                e2=np.sum(e**2)**0.5+0.00001
                w[x*np.shape(Xp)[1]+y]=(e2**(r-2))
                Xp[x,y,0,:,:,:]+=(z[x*np.shape(Xp)[1]+y]-Xp[x,y,0,:,:,:])*w[x*np.shape(Xp)[1]+y]
                overlap=patch_size[0]-subsampling_gap
                avgx=np.copy(Xp[x,y,0,:overlap,:,:])
                avgy=np.copy(Xp[x,y,0,:,:overlap,:])
                if(x>0):
                    avgx+=Xp[x-1,y,0,-overlap:,:,:]
                    avgx/=2.0
                if(y>0):
                   avgy+=Xp[x,y-1,0,:,-overlap:,:]
                   avgy/=2.0
                Xp[x,y,0,:overlap,:,:]=avgx
                Xp[x,y,0,:,:overlap,:]=avgy
                 
def segement(image):

    mask = np.zeros(image.shape[:2], np.uint8)
    backgroundModel = np.zeros((1, 65), np.float64) 
    foregroundModel = np.zeros((1, 65), np.float64) 

    img_height=image.shape[0]
    img_width= image.shape[1]
    rectangle = (1,1,img_width,img_height)

    cv2.grabCut(image, mask, rectangle, backgroundModel, foregroundModel, 10, cv2.GC_INIT_WITH_RECT) 

    mask_forground = np.where((mask == 2)|(mask == 0), 0, 1).astype('uint8') 
    result_img = image * mask_forground[:, :, np.newaxis] 
    result_mask = np.where((result_img>0),0.6,0).astype(float)

    return result_mask

def color_transfer(content,style):
    content=cv2.cvtColor(content, cv2.COLOR_BGR2LAB)
    style=cv2.cvtColor(style, cv2.COLOR_BGR2LAB)    
    content[:,:,0]-=np.mean(content[:,:,0])
    content[:,:,1]-=np.mean(content[:,:,1])
    content[:,:,2]-=np.mean(content[:,:,2])
    content[:,:,0]*=(np.std(style[:,:,0])/(np.std(content[:,:,0])+0.00001))
    content[:,:,1]*=(np.std(style[:,:,1]/(np.std(content[:,:,1])+0.00001)))
    content[:,:,2]*=(np.std(style[:,:,2]/(np.std(content[:,:,2])+0.00001)))
    content[:,:,0]+=np.mean(style[:,:,0])
    content[:,:,1]+=np.mean(style[:,:,1])
    content[:,:,2]+=np.mean(style[:,:,2])
    return cv2.cvtColor(content.astype(np.float32), cv2.COLOR_LAB2BGR)

def style_transfer(content, style, r, L, Iirls, patch_sizes, subsampling_gaps, Ialg, seg_mask):
    content = color_transfer(content,style)
    show_images([content],["content"])
    content_pyramid_tuple = tuple(pyramid_gaussian(content, channel_axis=-1, max_layer=L, downscale=2))
    style_pyramid_tuple = tuple(pyramid_gaussian(style, channel_axis=-1, max_layer=L, downscale=2))
    w_pyramid_tuple = tuple(pyramid_gaussian(seg_mask, channel_axis=-1, max_layer=L, downscale=2))
    
    content_pyramid = []
    style_pyramid = []
    w_pyramid = []  
    for i in range (0,L):
        content_pyramid.append(content_pyramid_tuple[i])
        style_pyramid.append(style_pyramid_tuple[i])
        w_pyramid.append(w_pyramid_tuple[i])
    X = np.copy(content_pyramid[L-1])
    
    X = np.pad(X, ((0, patch_sizes[0]), (0, patch_sizes[0]), (0, 0)), mode='reflect')
    resized_style = cv2.resize(style, (np.shape(X)[1],np.shape(X)[0])) 
    X[-patch_sizes[0]:np.shape(X)[0], :, :] = resized_style[-patch_sizes[0]:np.shape(X)[0], :, :]
    X[:-patch_sizes[0], -patch_sizes[0]:, :] = resized_style[:-patch_sizes[0], -patch_sizes[0]:, :]

    for l in range (L-1,-1,-1):
        if(l == L-1):
            X= random_noise(X, mode="gaussian",mean=0,var=50)
        else:
            X= random_noise(X, mode="gaussian",mean=0,var=0.01)
        content_pyramid[l] = np.pad(content_pyramid[l], ((0, patch_sizes[0]), (0, patch_sizes[0]), (0,0)), mode='constant', constant_values=(0,0))
        w_pyramid[l] = np.pad(w_pyramid[l], ((0, patch_sizes[0]), (0, patch_sizes[0]), (0, 0)), mode='constant', constant_values=(0,0))
        for s in range(0,len(patch_sizes)):
            style_patches =  extract_patches(style_pyramid[l],(patch_sizes[s], patch_sizes[s], 3),subsampling_gaps[s])
            flatten_style_patches = style_patches.reshape(-1, patch_sizes[s] * patch_sizes[s] * 3)
            nn_model = NearestNeighbors(n_neighbors=1).fit(flatten_style_patches)    
            for k in range(0,Ialg):
                z=[]
                Xp=extract_patches(X,(patch_sizes[s], patch_sizes[s], 3),subsampling_gaps[s])
                flatten_Xp = Xp.reshape(-1, patch_sizes[s] * patch_sizes[s] * 3)

                sc=0
                for Xpatch in flatten_Xp:
                    flatten_Xp = Xpatch.reshape(-1, patch_sizes[s] * patch_sizes[s] * 3)
                    unflattened_Xp=flatten_Xp.reshape(patch_sizes[s], patch_sizes[s], 3)
                    if(sc%Xp.shape[0]):
                        unflattened_Xp[:,:patch_sizes[s]-subsampling_gaps[s],:]=z[sc-1][:,-(patch_sizes[s]-subsampling_gaps[s]):,:]
                    if(sc>=Xp.shape[1]):
                        unflattened_Xp[:patch_sizes[s]-subsampling_gaps[s],:,:]=z[sc-Xp.shape[1]][-(patch_sizes[s]-subsampling_gaps[s]):,:,:]

                    # show_images([unflattened_Xp])
                    flatten_Xp = unflattened_Xp.reshape(-1, patch_sizes[s] * patch_sizes[s] * 3)

                    flatten_neighbour_patch = flatten_style_patches[nn_model.kneighbors(flatten_Xp)[1][0][0]]
                    z.append(flatten_neighbour_patch.reshape(patch_sizes[s], patch_sizes[s], 3))
                    sc+=1
                irls(X,z,r,Iirls,(patch_sizes[s],patch_sizes[s],3),subsampling_gaps[s]) 
                X=color_transfer(X.astype(np.float32),style_pyramid[l].astype(np.float32))
                X =((1.0-w_pyramid[l])* X).astype(np.float32) + ((w_pyramid[l].astype(np.float32))*content_pyramid[l]).astype(np.float32)
                X = cv2.bilateralFilter(X, 1, sigmaColor=5, sigmaSpace=10) 
                show_images([X])
        if (l>0) : 
            padding_down=cv2.resize(X[-patch_sizes[0]:np.shape(X)[0], :, :], (np.shape(content_pyramid[l-1])[1]+patch_sizes[0], patch_sizes[0]))   
            padding_right = cv2.resize(X[:-patch_sizes[0], -patch_sizes[0]:, :], (patch_sizes[0], np.shape(content_pyramid[l-1])[0]))

            X = X[0:-patch_sizes[0], 0:-patch_sizes[0], :]
            X = cv2.resize(X, (np.shape(content_pyramid[l-1])[1], np.shape(content_pyramid[l-1])[0]))   
            X = np.pad(X, ((0, patch_sizes[0]), (0, patch_sizes[0]), (0, 0)), mode='constant', constant_values=(1,1))
            X[-patch_sizes[0]:np.shape(X)[0], :, :] = padding_down
            X[:-patch_sizes[0], -patch_sizes[0]:, :] = padding_right
    return X

X=io.imread("imgs/russia.jpg").astype(np.float32)/255
style=io.imread("imgs/papers.co-ar55-vicent-van-gogh-starry-night-art-classic-29-wallpaper.jpg").astype(np.float32)/255
# con = io.imread("imgs/russia.jpg")
# seg_mask = segement(con)
seg_mask = io.imread("imgs/seg5.jpg").astype(np.float32)/255
# io.imsave("seg.jpg", seg_mask) 
# iio.imwrite(uri="D:\ip_project\style-transfer\imgs\seg5.jpg", image=(seg_mask*255).astype(np.uint8))
style = cv2.resize(style, (np.shape(X)[0], np.shape(X)[1]))
show_images([style])
patches_sizes = [50, 40]
sampling_gaps = [40, 30]
r = 0.8
L_max = 3
I_irls = 7
I_alg = 3
out = style_transfer(X,style,r,L_max,I_irls,patches_sizes,sampling_gaps,I_alg , seg_mask)
iio.imwrite(uri="imgs\output\content_styl18.jpg", image=(out[:-patches_sizes[0],:-patches_sizes[0],:]*255).astype(np.uint8))
show_images([X, style, out[:-patches_sizes[0],:-patches_sizes[0],:]])

In [ ]:
import networkx as nx


def graph_from_adjacency_matrix(adjacency_matrix):
    # Create a graph from the adjacency matrix
    G = nx.Graph()
    num_nodes = len(adjacency_matrix)

    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if adjacency_matrix[i][j] != 0:
                G.add_edge(i, j, weight=adjacency_matrix[i][j])

    return G

def find_min_cut(graph):
    # Perform the Stoer-Wagner min cut algorithm
    min_cut_value, partition = nx.stoer_wagner(graph)

    # Extract the cut set
    cut_set = set(partition[0])

    # Return the minimum cut value and the cut set
    return min_cut_value, cut_set

adjacency_matrix = np.array([   [0, 2, 0, 1],
                                [2, 0, 1, 1],
                                [0, 1, 0, 4],
                                [1, 1, 4, 0]])

# Create a graph from the adjacency matrix
G = graph_from_adjacency_matrix(adjacency_matrix)

print(list(G.neighbors(2)))

# Find the minimum cut
min_cut_value, cut_set = find_min_cut(G)

# Print the results
print("Minimum cut value:", min_cut_value)
print("Cut set:", cut_set)
